In [1]:
"""
Important Stuff
"""

# OpenCV is a leading open source library for computer vision. To use it, we need to import it
# The easiest way to install OpenCV is by installing Anaconda distribution of python available for free here:
# https://www.anaconda.com/distribution/
import cv2  

# We need two images seperated in time to have movement
img_initial = None
img_final = None

# Webcam index, this should be 0 for most laptops
DEVICE = 0

# We shrink outputput from the webcam in order to achieve a higher FPS
WIDTH = 640
HEIGHT = 480

"""
User Interface Stuff (You can ignore this)
"""
import IPython
import ipywidgets as widgets
from IPython.display import display

ipython = IPython.get_ipython()

style = {'description_width': 'initial'}

w_image = widgets.Image(width=WIDTH, height=HEIGHT, format='png',
                        layout=widgets.Layout(width='100%'))
w_min_area_coefficient = widgets.FloatSlider(min=0, max=1, value=0.026, step=0.01, 
                                             description='Minimum Bounding Box Area',
                                             layout=widgets.Layout(width='100%'),
                                             style=style)
w_thresh = widgets.IntSlider(min=0, max=255, value=32, 
                             description='Threshold',
                             layout=widgets.Layout(width='100%'),
                             style=style)
w_dilate = widgets.IntSlider(min=0, max=20, value=8,
                             description='Dilate',
                             layout=widgets.Layout(width='100%'),
                             style=style)
w_drop = widgets.Dropdown(options=[
    'Original Image',
    'Greyscale',
    'Gaussian Blur',
    'Difference Image',
    'Threshold',
    'Dilate'
], style=style)

w_bbox = widgets.Checkbox(
    value=True,
    description='Show bounding boxes',
    style=style
)
vbox = widgets.VBox([w_image, w_drop, w_bbox, w_thresh, w_dilate, w_min_area_coefficient])

display(vbox)

In [2]:
cap = cv2.VideoCapture(DEVICE)

try:
    while True:
        # Have to call this to get update values from sliders / dropdowns
        ipython.kernel.do_one_iteration()
        
        # Read the frame from the camera
        ret, frame = cap.read()
        
        # Resize the frame to a lower resolution for performance reasons
        img_original = cv2.resize(frame, (WIDTH, HEIGHT))

        # Convert image to greyscale
        img_grey = cv2.cvtColor(img_original, cv2.COLOR_BGR2GRAY)
        
        # Apply a slight guassian blur (low pass filter) to filter out noise
        img_blur = cv2.GaussianBlur(img_grey, (3, 3), 0)

        img_final = img_blur
        
        # Only compute the difference image if we have both a current and previous frame
        if img_initial is not None:
            
            # By looking at the difference between our two images, we can find motion
            img_diff = cv2.absdiff(img_initial, img_final)

            # Quantizes pixels to only represent ON or OFF instead of a wide range of intensity values
            _, img_threshold = cv2.threshold(img_diff, w_thresh.value, 255, cv2.THRESH_BINARY)

            # Each ON pixel expands in every direction.
            img_dilate = cv2.dilate(img_threshold, None, iterations=w_dilate.value)
            
            # A contour is a group of "on" pixels that are connected to eachother
            # Here we represent contours by the lines that form their edges
            cnts, _ = cv2.findContours(img_dilate, cv2.RETR_EXTERNAL,
                                   cv2.CHAIN_APPROX_SIMPLE)
            
            
            # Visualization Code
            if w_drop.value == 'Original Image':
                draw_image = img_original
            elif w_drop.value == 'Greyscale':
                draw_image = img_grey
                draw_image = cv2.cvtColor(draw_image, cv2.COLOR_GRAY2BGR)
            elif w_drop.value == 'Gaussian Blur':
                draw_image = img_blur
                draw_image = cv2.cvtColor(draw_image, cv2.COLOR_GRAY2BGR)
            elif w_drop.value == 'Difference Image':
                draw_image = img_diff
                draw_image = cv2.cvtColor(draw_image, cv2.COLOR_GRAY2BGR)
            elif w_drop.value == 'Threshold':
                draw_image = img_threshold
                draw_image = cv2.cvtColor(draw_image, cv2.COLOR_GRAY2BGR)
            elif w_drop.value == 'Dilate':
                draw_image = img_dilate
                draw_image = cv2.cvtColor(draw_image, cv2.COLOR_GRAY2BGR)

            
            # Iterate through every contour we found and determine if it should get a bounding box
            for c in cnts:
                # If the contour is too small, ignore it
                if cv2.contourArea(c) < int(w_min_area_coefficient.value * WIDTH * HEIGHT):
                    continue

                # Convert the contour to a rectangle starting at (x, y) with a width w and height h
                (x, y, w, h) = cv2.boundingRect(c)
                
                # Visualization Code
                if w_bbox.value:
                    cv2.rectangle(draw_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                
            # Visualization Code
            result, img_png = cv2.imencode('.png', draw_image)
            w_image.value = img_png.tobytes()

        img_initial = img_final

        
except KeyboardInterrupt:
    pass
finally:
    cap.release()
